# Initial Experimentation

## Web References

### Bitcoin Address

- [Bitcoin Address](https://learnmeabitcoin.com/technical/address)
- [Bitcoin Checksum](https://learnmeabitcoin.com/technical/checksum)
- [How do I check the checksum of a Bitcoin address?](https://bitcoin.stackexchange.com/questions/32353/how-do-i-check-the-checksum-of-a-bitcoin-address)
- [Base58Check encoding](https://en.bitcoin.it/wiki/Base58Check_encoding#Creating_a_Base58Check_string)

In [126]:
import unittest
import hashlib
from cryptography.hazmat.primitives import hashes

## Transaction Class

In [127]:
class TestTransaction(unittest.TestCase):
    def setUp(self):
        self.tx = Transaction(
            sender_hash=bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d"),
            recipient_hash=bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca"),
            sender_public_key=bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
                                            "03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
                                            "b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
                                            "31373afe8d1c860a9ac47d8e2659b74d437435b0" +
                                            "5f2c55bf3f033ac1"),
            amount=10,
            fee=2,
            nonce=5,
            signature=bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
                                    "2f173378cf78cf79c7978a2337fbad141d022100" +
                                    "ec27704d4d604f839f99e62c02e65bf60cc93ae1"
                                    "735c1ccf29fd31bd3c5a40ed"),
            txid=bytes.fromhex("ca388e0890b71bd1775460d478f26af3776c9b4f" +
                                "6c2b936e1e788c5c87657bc3"))

    def test_verify_sender_hash(self):
        """
        Test that the sender_hash is verified correctly.
        """
        # test a valid hash
        self.tx.sender_hash = bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d")
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)

        # test a sender_hash that do not match the public key
        self.tx.sender_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        # test an invalid hash
        self.tx.sender_hash = bytearray(b'thequickbrownfox')
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

    def test_verify_recipient_hash(self):
        """
        Test that the recipient_hash is verified correctly.
        """
        # test a valid hash
        self.tx.recipient_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)            

        # test an invalid hash
        self.tx.recipient_hash = bytearray(b'thequickbrownfox')
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

    def test_verify_amount(self):
        """
        Test that the amount is verified correctly.
        """
        # test a valid amount
        self.tx.amount = 50
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)     

        # the amount should be a whole number
        self.tx.amount = 50.1
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        # the amount should be between 1 and sender_balance
        self.tx.amount = 0
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        self.tx.amount = -10
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)   

        self.tx.amount = 101
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)           

    def test_verify_fee(self):
        """
        Test that the fee is verified correctly.
        """
        self.tx.amount = 50

        # test a valid fee
        self.tx.fee = 5
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)

        # the fee can be zero
        self.tx.fee = 0
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)    

        # the fee should be a whole number
        self.tx.fee = 10.1
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        # the fee should be between 0 and amount
        self.tx.fee = -10
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)   

        self.tx.fee = 51
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

    def test_verify_nonce(self):
        """
        Verify that the nonce is a valid value.
        """
        self.tx.nonce = 10

        # test a valid nonce
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=9)

        # the nonce should be sender_previous_nonce + 1
        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=8)

        with self.assertRaises(ValueError):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=11)

    def test_create_txid(self):
        """
        Test Create a transaction id.
        """
        self.assertEqual(self.tx.create_txid(), self.tx.txid)

        # the transaction should not match if one of the values differ
        self.tx.fee = 3
        self.assertNotEqual(self.tx.create_txid(), self.tx.txid)

In [128]:
class Transaction:
    def __init__(self, sender_hash:bytes, recipient_hash:bytes, sender_public_key:bytes, amount:int, fee:int, nonce:int, signature, txid):
        self.sender_hash = sender_hash
        self.recipient_hash = recipient_hash
        self.sender_public_key = sender_public_key
        self.amount = amount
        self.fee = fee
        self.nonce = nonce
        self.signature = signature
        self.txid = txid
        #self.hash = self.calculate_hash()

    def verify(self, sender_balance:int, sender_previous_nonce:int):
        # sender and recipient validation
        if len(self.sender_hash) != 20:
            raise ValueError('Invalid sender hash')
        if len(self.recipient_hash) != 20:
            raise ValueError('Invalid recipient hash')

        # validate the sender hash
        if self.sender_hash != hashlib.sha1(self.sender_public_key).digest():
            raise ValueError('Invalid sender hash')

        # amount validation
        if self.amount < 1:
            raise ValueError('Invalid amount')
        if self.amount > sender_balance:
            raise ValueError('Insufficient funds')
        if float(self.amount).is_integer() == False:
            raise ValueError('The amount should be a whole number')

        # fee validation
        if self.fee < 0:
            raise ValueError('Invalid fee')
        if self.fee > self.amount:
            raise ValueError('The fee should be less than the amount')
        if float(self.fee).is_integer() == False:
            raise ValueError('The fee should be a whole number')

        # nonce validation
        if self.nonce <= sender_previous_nonce:
            raise ValueError('Invalid nonce')
        if self.nonce > sender_previous_nonce + 1:
            raise ValueError('Invalid nonce')
        
    def create_txid(self):
        """
        Create a hash of the transaction.
        """
        # create a hash of the transaction
        txid = hashes.Hash(hashes.SHA256())
        txid.update(self.sender_hash)
        txid.update(self.recipient_hash)
        txid.update(self.sender_public_key)
        txid.update(self.amount.to_bytes(8, byteorder = 'little', signed = False))
        txid.update(self.fee.to_bytes(8, byteorder = 'little', signed = False))
        txid.update(self.nonce.to_bytes(8, byteorder = 'little', signed = False))
        txid.update(self.signature)

        return txid.finalize()

unittest.main(argv=['ignored', '-v'], exit=False)

test_create_txid (__main__.TestTransaction)
Test Create a transaction id. ... ok
test_verify_amount (__main__.TestTransaction)
Test that the amount is verified correctly. ... ok
test_verify_fee (__main__.TestTransaction)
Test that the fee is verified correctly. ... ok
test_verify_nonce (__main__.TestTransaction)
Verify that the nonce is a valid value. ... ok
test_verify_recipient_hash (__main__.TestTransaction)
Test that the recipient_hash is verified correctly. ... ok
test_verify_sender_hash (__main__.TestTransaction)
Test that the sender_hash is verified correctly. ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.011s

OK


In [129]:
a_bytes = bytes('a', 'utf-8')
type(a_bytes)

bytes